# Corrective RAG

Self-reflection can enhance RAG, enabling correction of poor quality retrieval or generations.

Several recent papers focus on this theme, but implementing the ideas can be tricky.

Here we show how to implement self-reflective RAG using `Mistral` and `LangGraph`.

We'll focus on ideas from one paper, `Corrective RAG (CRAG)` [here](https://arxiv.org/pdf/2401.15884.pdf).

![Screenshot 2024-02-07 at 1.21.51 PM.png](attachment:a65940f9-5c51-4d7c-9ca1-ae576e4bb51a.png)

## Setup

### Using APIs 

* Set `MISTRAL_API_KEY` and set up Subscription to activate it.
* Set `TAVILY_API_KEY` to enable web search [here](https://app.tavily.com/sign-in).

### Using CoLab 

* [Here](https://colab.research.google.com/drive/1U5OcwWjoXZSud30q4XOk1UlIJNjaD3kX?usp=sharing) is a link to a CoLab for this notebook. 

### Running Locally 

If you want to run this locally (e.g., on your laptop), use [Ollama](https://ollama.ai/library/mistral/tags):

* Download [Ollama app](https://ollama.ai/).
* Download a `Mistral` model e.g., `ollama pull mistral:7b-instruct`, from various Mistral versions [here](https://ollama.ai/library/mistral) and Mixtral versions [here](https://ollama.ai/library/mixtral) available.
* Download LLaMA2 `ollama pull llama2:latest` to use Ollama embeddings.
* Set flags indicating we will run locally and the Mistral model downloaded:
  
```
run_local = "Yes"
local_llm = "mistral:7b-instruct"
```

### Tracing 

* Optionally, use [LangSmith](https://docs.smith.langchain.com/) for tracing (shown at bottom) by setting: 

```
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
export LANGCHAIN_API_KEY=<your-api-key>
```

## Indexing

First, let's index a popular blog post on agents using [Mistral embeddings](https://js.langchain.com/docs/integrations/text_embedding/mistralai).

We'll use a local vectorstore, [Chroma](https://js.langchain.com/docs/integrations/vectorstores/chroma).